In [1]:
import os
import sys
import math
import logging
import structlog
from pathlib import Path
import json

import tomli
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (12, 6.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

In [2]:
logging.basicConfig(level=logging.WARNING, stream=sys.stdout)

In [3]:
import pytanis
from pytanis import GSheetClient, PretalxClient
from pytanis.google import Scope, gsheet_rows_for_fmt
from pytanis.review import read_assignment_as_df, save_assignments_as_json, Col
from pytanis.pretalx import subs_as_df, reviews_as_df, speakers_as_df
from pytanis.utils import implode

In [4]:
# Be aware that this notebook might only run with the following version
pytanis.__version__ 

'0.3.post1.dev5+g3365eac.d20230228'

In [108]:
# Import event-specific settings to don't have them here in the notebook
with open('config.toml', 'rb') as fh:
    cfg = tomli.load(fh)


## Gather all Data, i.e. Submissions, Votes and Scheduling Data

In [6]:
pretalx_client = PretalxClient(blocking=True)
talks_count, talks = pretalx_client.submissions(cfg['event_name'], params={"questions": "all", "state": ["confirmed"]})
talks = list(talks)
rooms_count, rooms = pretalx_client.rooms(cfg['event_name'])
rooms = list(rooms)

  0%|          | 0/112 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
talks_df = subs_as_df(talks, with_questions=True)

In [8]:
# Ignore Keynotes when scheduling
talks_df = talks_df.loc[~(talks_df[Col.submission_type] == "Keynote")]

In [9]:
talk2idx = {v: k for k, v in enumerate(set(talks_df[Col.submission].to_list()))}
idx2talk = {v: k for k, v in talk2idx.items()}
float2int_factor = 4 # reduces precision loss when converting from float to int

In [10]:
votes_df = pd.read_csv("./pyconde-pydata-berlin-2023-public-votes.csv")
votes_df = votes_df.rename(columns={'code': Col.submission, 'score': Col.vote_score})
# Remove votes for submissions that wheren't accepted
votes_df = votes_df.loc[votes_df[Col.submission].apply(lambda talk: talk in talk2idx.keys())].reset_index(drop=True)
# Remove votes equal 1 as this meant "being indifferent"
votes_df = votes_df.loc[votes_df[Col.vote_score] > 1]
# Move a score of 2 to 1, so that "must see" counts tripple compared to "want to see"
votes_df.loc[votes_df[Col.vote_score] == 2, Col.vote_score] = 1

In [11]:
talk_lengths = {s:d for _, (s, d) in talks_df[[Col.submission, Col.duration]].iterrows()}
# following talks build sequences and thus need to be in the same session and after one another
consecutive_tutorials = [("VFZ3VT", "DEQM3J"), ("KUKU9Z", "YWTRSG")]

In [12]:
pops_dict = votes_df.groupby(Col.submission)['Vote Score'].sum().to_dict()
# give sponsored talks the median popularity
for talk in set(talks_df[Col.submission].to_list()) - set(pops_dict.keys()):
    pops_dict[talk] = np.median(list(pops_dict.values()))
# normalize the popularities
min_pop = np.min(list(pops_dict.values()))
pops_dict = {k:int(float2int_factor*v / min_pop) for k, v in pops_dict.items()}
print(pops_dict)

{'3TH9UC': 8, '77MWVW': 5, '7EUPC3': 8, '7FTL7H': 11, '7NW7JC': 5, '7SYVML': 7, '8CVQDW': 14, '8VYHKG': 5, '9Q38VT': 8, '9SENVW': 10, 'A7B8P8': 9, 'AKXXQD': 10, 'AQAJDH': 9, 'AUJYP7': 7, 'AXMS87': 4, 'B7PCUR': 6, 'B8FKHC': 12, 'CBHYXG': 10, 'CHLT3D': 8, 'CTKC7B': 4, 'DB3KC7': 23, 'DECAHT': 8, 'DXMXZH': 9, 'E77G9H': 7, 'EAKYPL': 6, 'ENQBPJ': 9, 'FVWF7R': 7, 'FZY9VV': 11, 'G9TATQ': 8, 'GBYWCY': 20, 'GCPNMT': 9, 'GLQH8X': 5, 'GXAKV8': 12, 'GXPWJ8': 5, 'GYEZVW': 10, 'H7ZCWK': 12, 'H8KMTT': 8, 'HLMGHX': 7, 'HMGCPL': 4, 'HNKMMP': 9, 'J9KRKZ': 19, 'JY3R3Z': 8, 'KUKU9Z': 13, 'KYLLZA': 5, 'LCCGTT': 14, 'LMGF8V': 18, 'LXBGZS': 10, 'M7XMFB': 10, 'MBZJE9': 15, 'MJRFLC': 5, 'MLAGKM': 10, 'MQHTHY': 11, 'MTRFT3': 5, 'MTXCHH': 7, 'NFYQHB': 13, 'NLFFSE': 5, 'NUF87W': 16, 'NWSLUH': 8, 'PEQZTC': 5, 'PPXA79': 7, 'PQZR3Q': 10, 'Q9GVEK': 6, 'QLCNN9': 6, 'QUAXG3': 4, 'RDQH3W': 11, 'RPMMKZ': 10, 'RQ3MWN': 5, 'S79HEH': 22, 'S8GYFF': 7, 'SJCEFG': 12, 'SSTCTS': 8, 'SVXFP8': 4, 'TCWCVV': 7, 'TP7ABB': 14, 'TWPBZF'

In [13]:
room_caps_dict = {room.name.en:room.capacity for room in rooms}
print(room_caps_dict)
# normalize room_caps like popularity by dividing by min
min_room_cap = np.min(list(room_caps_dict.values()))
room_caps_dict = {k:int(float2int_factor*v / min_room_cap) for k, v in room_caps_dict.items()}
tutorial_rooms =  {'A03-A04', 'A05-A06'}
talk_rooms = set(room_caps_dict.keys()) - tutorial_rooms
print(room_caps_dict)

{'Kuppelsaal': 950, 'B09': 240, 'B07-B08': 200, 'B05-B06': 300, 'A1': 80, 'A03-A04': 140, 'A05-A06': 140}
{'Kuppelsaal': 47, 'B09': 12, 'B07-B08': 10, 'B05-B06': 15, 'A1': 4, 'A03-A04': 7, 'A05-A06': 7}


In [14]:
# generate cooccurrence penalty for all talks scheduled at the same time-slot & room using Gram-matrix
n_users = votes_df['voter'].nunique()
n_talks = len(talks_df)

def user_interaction(user_votes):
    x = np.zeros(n_talks)
    for _, (sub, vote_score) in user_votes[[Col.submission, Col.vote_score]].iterrows():
        x[talk2idx[sub]] = vote_score
    return x

X = np.vstack(votes_df.groupby("voter").apply(user_interaction))
assert X.shape == (n_users, n_talks)

cooccurance_penalty = X.T @ X

## Time-Table Scheduling

We consider following constraints:
* each talk must be assigned once
* each room/timeslot combination can only be occupied by one talk at most
* the length of the timeslot must match the length of the talk
* some tutorials have part 1 & 2,thus need to be consecutive

We optimize an objective which considers:
* the preferences for day and time of the speakers are considered (if they provided some)
* the more popular a talk is, the more capacity the assigned room should have
* if many people are interested in seeing two talks (data from the votes) these talks should rather not be scheduled in parallel

In [15]:
import pyomo.environ as pyo
import pyomo.gdp as pyogdp
from pyomo.contrib.appsi.solvers import Highs
from itertools import product, combinations

In [93]:
model = pyo.ConcreteModel(name="PyConDE/PyData Schedule")

##############
# Index Sets #
##############

model.sTalks = pyo.Set(initialize=talks_df[Col.submission].values, ordered=True)
model.sDays = pyo.Set(initialize=["Monday", "Tuesday", "Wednesday"], ordered=True)
model.sSessions = pyo.Set(initialize=["Morning 1", "Morning 2", "Afternoon 1", "Afternoon 2"], ordered=True)
model.sSlots = pyo.Set(initialize=["First", "Second", "Third"], ordered=True)
model.sRooms = pyo.Set(initialize=room_caps_dict.keys(), ordered=True)
model.sConsecTutorials = pyo.Set(initialize=consecutive_tutorials, dimen=2)

##############
# Parameters #
##############

def init_slot_preference(model, t, d, s, l, r):
    high_pref = 1000
    if t in {"WHACAT", "MYARJG", "E77G9H", "8WXSR9"} and d == "Monday":
        return high_pref
    elif t in {"VBP3PE", "TP7ABB"} and d == "Monday" and s.startswith("Morning"):
        return high_pref
    elif t in {"KYLLZA"} and d == "Tuesday":
        return high_pref
    elif t in {"9Q38VT"} and d in {"Monday", "Tuesday"}:
        return high_pref
    else:
        return 0

model.pPreferences = pyo.Param(model.sTalks, model.sDays, model.sSessions, model.sSlots, model.sRooms, initialize=init_slot_preference, mutable=True)
model.pPopularity = pyo.Param(model.sTalks, initialize=pops_dict, mutable=False)
model.pRoomCapacities = pyo.Param(model.sRooms, initialize=room_caps_dict, mutable=False)
model.pCoOccurences = pyo.Param(model.sTalks, model.sTalks, initialize={(t1, t2): cooccurance_penalty[talk2idx[t1], talk2idx[t2]] for t1, t2 in combinations(model.sTalks, 2)})

def init_slot_length(model, d, s, l, r):
    if d == "Monday":
        if s == "Morning 1":
            if r in talk_rooms:
                if l in {"First", "Second"}:
                    return 45
                else:
                    return 0 # no Third slot
            else:
                return 90 if l == "First" else 0
        elif s == "Morning 2" or s == "Afternoon 1":
            return 0 # does not exist
        else: # "Afternoon 2":
            if r in talk_rooms:
                if r == "Kuppelsaal":
                    return 0
                elif l in {"First", "Second"}:
                    return 30
                else:
                    return 0 # no Third slot
            else:
                return 90 if l == "First" else 0
    elif d == "Tuesday":
        if s == "Morning 1" or s == "Afternoon 2":
            if r in talk_rooms:
                return 30
            else:
                return 90 if l == "First" else 0
        elif s == "Afternoon 1":
            if r in talk_rooms:
                if l == "First":
                    return 30
                elif l == "Second":
                    return 45
                else:
                    return 0 # no Third slot
            else:
                return 90 if l == "First" else 0
        else: # Morning 2
            return 0
    else: # Wednesday
        if s == "Morning 1":
            if r in talk_rooms:
                if l == "First":
                    return 45
                elif l == "Second":
                    return 30
                else:
                    return 0 # no Third slot
            else:
                return 90 if l == "First" else 0
        elif s == "Morning 2":
            if r in talk_rooms:
                if l in {"First", "Second"}:
                    return 30
                else:
                    return 0 # no Third slot
            else:
                return 90 if l == "First" else 0
        elif s == "Afternoon 1":
            if r in talk_rooms:
                return 30
            else:
                return 90 if l == "First" else 0
        else: # Afternoon 2
            return 0
    raise RuntimeError(f"Unhandled case {d}, {s}, {l}, {r}")
    
model.pSlotLengths = pyo.Param(model.sDays, model.sSessions, model.sSlots, model.sRooms, initialize=init_slot_length, mutable=False)
model.pTalkLengths = pyo.Param(model.sTalks, initialize=talk_lengths, mutable=False)

#############
# Variables #
#############

## Auxilliary variables
model.vbTalkRoom = pyo.Var(model.sTalks, model.sRooms)
model.vbCoOccurences = pyo.Var(model.sTalks, model.sTalks, model.sDays, model.sSessions, model.sSlots, initialize=0., domain=pyo.Binary)
model.vnnrPopRoomCapLeft = pyo.Var(model.sTalks, domain=pyo.NonNegativeReals)
model.vnnrPopRoomCapRight = pyo.Var(model.sTalks, domain=pyo.NonNegativeReals)

## Decision variable
model.vbSchedule = pyo.Var(model.sTalks, model.sDays, model.sSessions, model.sSlots, model.sRooms, domain=pyo.Binary)

###############
# Constraints #
###############

# Make sure talk lengths fits slot lengths
model.ctTalkSlotFit = pyo.ConstraintList()
for t in model.sTalks:
    model.ctTalkSlotFit.add(sum(model.vbSchedule[t, d, s, l, r] * model.pSlotLengths[d, s, l, r] for d, s, l, r in product(model.sDays, model.sSessions, model.sSlots, model.sRooms)) == model.pTalkLengths[t])
    
# Make sure each room/timeslot-combination is occupied only with one talk at most
model.ctTimeRoomOccup = pyo.ConstraintList()
for d, s, l, r in product(model.sDays, model.sSessions, model.sSlots,  model.sRooms):
    model.ctTimeRoomOccup.add(sum(model.vbSchedule[:, d, s, l, r]) <= 1)
    
# Make sure each talk is assigned once
model.ctTalkAssigned = pyo.ConstraintList()
for t in model.sTalks:
    model.ctTalkAssigned.add(sum(model.vbSchedule[t, :, :, :, :]) == 1)
model.ctCoOccurences = pyo.ConstraintList()
for d, s, l in product(model.sDays, model.sSessions, model.sSlots):
    for t1, t2 in combinations(model.sTalks, 2):
        model.ctCoOccurences.add(model.vbCoOccurences[t1, t2, d, s, l] + 1 >= model.vbSchedule[t1, d, s, l, r] + model.vbSchedule[t2, d, s, l, r])

# Set vbTalkRoom auxilliary variable
model.ctTalkRoom = pyo.ConstraintList()
for t, r in product(model.sTalks, model.sRooms):
    model.ctTalkRoom.add(sum(model.vbSchedule[t, ..., r]) == model.vbTalkRoom[t, r])

# Penalty helper terms for absolute value of deviations from room capacity to popularity
model.ctPopularityRoomCapacity = pyo.ConstraintList()
for t in model.sTalks:
    model.ctPopularityRoomCapacity.add(model.vnnrPopRoomCapLeft[t] - model.vnnrPopRoomCapRight[t] == sum(model.vbTalkRoom[t, r]*(model.pRoomCapacities[r] - model.pPopularity[t]) for r in model.sRooms))

# Enforce that some talks need to be in consecutive order
def ct_consecutive_slots(model, t1, t2):
    possibilities = [(("Tuesday", "Afternoon 1", "First", r), ("Tuesday", "Afternoon 2", "First", r)) for r in tutorial_rooms]
    possibilities.extend([(("Wednesday", "Morning 1", "First", r), ("Wednesday", "Morning 2", "First", r)) for r in tutorial_rooms])
    return [model.vbSchedule[t1, d1, s1, l1, r1] + model.vbSchedule[t2, d2, s2, l2, r2] >= 2 for (d1, s1, l1, r1), (d2, s2, l2, r2) in possibilities]
    
model.disjConsecutiveTutorials = []
for t1, t2 in consecutive_tutorials:
    model.disjConsecutiveTutorials.append(pyogdp.Disjunction(expr=ct_consecutive_slots(model, t1, t2)))
    
#############
# Objective #
#############   
def objective(model):
    preference_term = pyo.dot_product(model.pPreferences, model.vbSchedule)
    pop_roomcap_term = sum(model.vnnrPopRoomCapLeft[t] + model.vnnrPopRoomCapRight[t] for t in model.sTalks)
    cooccurance_terms = []
    for d, s, l in product(model.sDays, model.sSessions, model.sSlots):
        cooccurance_terms.append(sum(model.vbCoOccurences[t1, t2, d, s, l] * model.pCoOccurences[t1, t2] for t1, t2 in combinations(model.sTalks, 2)))
    cooccurance_term = sum(cooccurance_terms) + sum(model.vbCoOccurences[...]) # last term forces vbCoOccurences to 0 if possible

    return preference_term - pop_roomcap_term - cooccurance_term
    
model.obj = pyo.Objective(sense=pyo.maximize, rule=objective)
pyo.TransformationFactory("gdp.bigm").apply_to(model)


In [94]:
# consistency check: amount of lengths of talks/timeslots must match
print("Slots:\n", pd.Series(model.pSlotLengths.values()).value_counts(), sep='')
print("\nTalks:\n", talks_df[Col.duration].value_counts(), sep='')

Slots:
0     143
30     73
45     20
90     16
dtype: int64

Talks:
30    73
45    20
90    16
Name: Duration, dtype: int64


In [95]:
# Optionally write out the model file to solve it on the CLI directly
#model.write(filename = "pyconde_pydata_schedule_20230301-2.mps", io_options = {"symbolic_solver_labels":True})

In [96]:
solver = Highs()
solver.config.time_limit = 600. # 10 minutes

In [97]:
sol = solver.solve(model)

Running HiGHS 1.5.1 [date: 2023-02-24, git hash: n/a]
Copyright (c) 2023 HiGHS under MIT licence terms


In [135]:
schedule_df = pd.DataFrame([idx for idx, val in model.vbSchedule.items() if val() >= 0.5], columns=[Col.submission, "Day", "Session", "Slot", "Room"])
schedule_df = schedule_df.loc[:, ["Day", "Session", "Slot", "Room", Col.submission]]
schedule_df["Day"] = pd.Categorical(schedule_df["Day"], model.sDays.data())
schedule_df["Session"] = pd.Categorical(schedule_df["Session"], model.sSessions.data())
schedule_df["Slot"] = pd.Categorical(schedule_df["Slot"], model.sSlots.data())
schedule_df["Room"] = pd.Categorical(schedule_df["Room"], model.sRooms.data())
schedule_df.sort_values(list(schedule_df.columns[:-1]), inplace=True)
schedule_df.reset_index(drop=True, inplace=True)
timetable_df = schedule_df.pivot(index=["Day", "Session", "Slot"], columns="Room", values=Col.submission)

In [136]:
timetable_df

Room                         Kuppelsaal     B09 B07-B08 B05-B06      A1  \
Day       Session     Slot                                                
Monday    Morning 1   First      KMGYZF  RDQH3W  WAVRYZ  DTBTVF  XDRNQC   
                      Second     VBP3PE  X89787  Q7GS8Y  GXAKV8  HMGCPL   
          Afternoon 2 First         NaN  WHACAT  MYARJG  XEVGVJ  UECWHD   
                      Second        NaN  8WXSR9  MLAGKM  E77G9H  MJRFLC   
Tuesday   Morning 1   First      GBYWCY  CHLT3D  VXPFFP  A7B8P8  RQ3MWN   
                      Second     SSTCTS  U7WAQW  9SENVW  YTHXML  WLMDZ7   
                      Third      8CVQDW  DXMXZH  7EUPC3  NFYQHB  AXMS87   
          Afternoon 1 First      AQAJDH  JY3R3Z  GCPNMT  9Q38VT  KYLLZA   
                      Second     DB3KC7  DECAHT  LXBGZS  B8FKHC  7NW7JC   
          Afternoon 2 First      J9KRKZ  H7ZCWK  TGZFSF  ENQBPJ  MTRFT3   
                      Second     MQHTHY  7FTL7H  CBHYXG  UNA9AN  QUAXG3   
                      Third      NWSLUH  SJCEFG  GYEZVW  3TH9UC  GXPWJ8   
Wednesday Morning 1   First      TCWCVV  W9HLK3  FVWF7R  WMAXSV  77MWVW   
                      Second     S79HEH  H8KMTT  AUJYP7  Y9TAMP  WMWZQC   
          Morning 2   First      Z8PESY  QLCNN9  HLMGHX  TP7ABB  8VYHKG   
                      Second     NUF87W  7SYVML  AKXXQD  LCCGTT  SVXFP8   
          Afternoon 1 First      ZRAFKA  B7PCUR  ABNXHC  PPXA79  NLFFSE   
                      Second     LMGF8V  G9TATQ  HNKMMP  MBZJE9  PEQZTC   
                      Third      TWPBZF  Q9GVEK  MTXCHH  VRGANP  CTKC7B   

Room                         A03-A04 A05-A06  
Day       Session     Slot                    
Monday    Morning 1   First   PQZR3Q  RPMMKZ  
                      Second     NaN     NaN  
          Afternoon 2 First   S8GYFF  DEQM3J  
                      Second     NaN     NaN  
Tuesday   Morning 1   First   MBH7GB  FZY9VV  
                      Second     NaN     NaN  
                      Third      NaN     NaN  
          Afternoon 1 First   M7XMFB  V9HBUU  
                      Second     NaN     NaN  
          Afternoon 2 First   UQ3KXD  KUKU9Z  
                      Second     NaN     NaN  
                      Third      NaN     NaN  
Wednesday Morning 1   First   XJSNBQ  GLQH8X  
                      Second     NaN     NaN  
          Morning 2   First   EAKYPL  ZYB38R  
                      Second     NaN     NaN  
          Afternoon 1 First   VFZ3VT  YWTRSG  
                      Second     NaN     NaN  
                      Third      NaN     NaN

## Upload to GSheet

In [137]:
# make submission code a hyperlink
schedule_df[Col.submission] = schedule_df[Col.submission].map(lambda sub: f'=HYPERLINK("https://pretalx.com/orga/event/{cfg["event_name"]}/submissions/{sub}", "{sub}")')
timetable_df = schedule_df.pivot(index=["Day", "Session", "Slot"], columns="Room", values=Col.submission)

In [138]:
gsheet_client = GSheetClient(read_only=False)

In [139]:
gsheet_client.save_df_as_gsheet(timetable_df.reset_index(), cfg['schedule_spread_id'], cfg['schedule_work_name'])

In [121]:
timetable_df

Room                                                                 Kuppelsaal  \
Day       Session     Slot                                                        
Monday    Morning 1   First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Afternoon 2 First                                                 NaN   
                      Second                                                NaN   
Tuesday   Morning 1   First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Third   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Afternoon 1 First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Afternoon 2 First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Third   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
Wednesday Morning 1   First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Morning 2   First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Afternoon 1 First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Third   =HYPERLINK("https://pretalx.com/orga/event/pyc...   

Room                                                                        B09  \
Day       Session     Slot                                                        
Monday    Morning 1   First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Afternoon 2 First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
Tuesday   Morning 1   First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Third   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Afternoon 1 First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Afternoon 2 First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Third   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
Wednesday Morning 1   First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Morning 2   First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second                                                NaN   
          Afternoon 1 First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Third   =HYPERLINK("https://pretalx.com/orga/event/pyc...   

Room                                                                    B07-B08  \
Day       Session     Slot                                                        
Monday    Morning 1   First   =HYPERLINK("https://pretalx.com/orga/event/pyc...   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
          Afternoon 2 First                                                 NaN   
                      Second  =HYPERLINK("https://pretalx.com/orga/event/pyc...   
Tuesday   Morn

## References
* https://fran-espiga.medium.com/mixed-integer-programming-for-time-table-scheduling-eee326deda75
* https://towardsdatascience.com/schedule-optimisation-using-linear-programming-in-python-9b3e1bc241e1
* https://math.stackexchange.com/questions/432003/converting-absolute-value-program-into-linear-program